In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings, HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits.conversational_retrieval.tool import create_retriever_tool
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
import json



In [2]:
docs = []
with open("final_data.json") as f:
    for line in f:
        row = json.loads(line)
        doc = Document(
            page_content = row["text"],
            metadata={"title": row["title"], "id": row["docid"], "source": row["text"]},
        )
        docs.append(doc)


In [3]:
docs[0]


Document(page_content="There are four main types of diabetic neuropathy. You can have one type or more than one type of neuropathy.Numbness or reduced ability to feel pain or temperature changes Tingling or burning feeling Sharp pains or cramps Muscle weakness Extreme sensitivity to touch — for some people, even a bedsheet's weight can be painful Serious foot problems, such as ulcers, infections, and bone and joint damage", metadata={'title': 'Diabetic neuropathy', 'id': 1, 'source': "There are four main types of diabetic neuropathy. You can have one type or more than one type of neuropathy.Numbness or reduced ability to feel pain or temperature changes Tingling or burning feeling Sharp pains or cramps Muscle weakness Extreme sensitivity to touch — for some people, even a bedsheet's weight can be painful Serious foot problems, such as ulcers, infections, and bone and joint damage"})

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs)


In [5]:
docs[0]


Document(page_content="There are four main types of diabetic neuropathy. You can have one type or more than one type of neuropathy.Numbness or reduced ability to feel pain or temperature changes Tingling or burning feeling Sharp pains or cramps Muscle weakness Extreme sensitivity to touch — for some people, even a bedsheet's weight can be painful Serious foot problems, such as ulcers, infections, and bone and joint damage", metadata={'title': 'Diabetic neuropathy', 'id': 1, 'source': "There are four main types of diabetic neuropathy. You can have one type or more than one type of neuropathy.Numbness or reduced ability to feel pain or temperature changes Tingling or burning feeling Sharp pains or cramps Muscle weakness Extreme sensitivity to touch — for some people, even a bedsheet's weight can be painful Serious foot problems, such as ulcers, infections, and bone and joint damage"})

In [43]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# self.embeddings = HuggingFaceEmbeddings(
#             model_name="pritamdeka/S-PubMedBert-MS-MARCO"
#         )
#         self.db = FAISS.load_local("symptomdb2.faiss", self.embeddings)


In [7]:
# db = FAISS.from_documents(docs, embeddings)
# db.save_local("symptomdb.faiss")


In [44]:
db = FAISS.load_local("symptomdb.faiss", embeddings)


In [9]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
import os


In [10]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [11]:
llm = ChatOpenAI(temperature=0)


# Query Augmentation

In [12]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

In [13]:
run_manager = CallbackManagerForRetrieverRun

In [48]:
# question = ["I have a headache and fever with a dry cough?", "headache, fever, dry cough"]
question = "I have a headache and fever with a dry cough?"

# QA_Docs = retriever.get_relevant_documents(query=question)
# len(QA_Docs)

In [30]:
from typing import List
from langchain import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")

class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)

output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. The last query should just be the 5 most important words of the original question.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""",
)
# llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
# question = "What are the approaches to Task Decomposition?"

# Run
retriever = MultiQueryRetriever(
    retriever=db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output
# Results
unique_docs = retriever.get_relevant_documents(
    query=question
)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the common symptoms of a sore throat and fever?', '2. How can I relieve a sore throat and fever?', '3. Are there any home remedies for a sore throat and fever?', '4. What are the possible causes of a sore throat and fever?', '5. Can you suggest any over-the-counter medications for a sore throat and fever?', '', 'sore throat, fever']


12

In [31]:
queries_list= llm_chain.run(question)

In [32]:
queries_list.lines

['1. What are the common symptoms of a sore throat and fever?',
 '2. How can I relieve a sore throat and fever?',
 '3. Are there any home remedies for a sore throat and fever?',
 '4. What are the possible causes of a sore throat and fever?',
 '5. Can you suggest any over-the-counter medications for a sore throat and fever?',
 '',
 'sore throat, fever']

# Vector Database Retrieval with augmentation from GPT

In [45]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())





In [49]:
query = question
result = chain({"question": query}, return_only_outputs=True)
scored_docs = db.similarity_search_with_relevance_scores(query, top_k=3)


In [50]:
result

{'answer': 'Based on the symptoms described (headache, fever, dry cough), it is possible that you may have the flu. However, it is important to consult with a healthcare professional for an accurate diagnosis and appropriate treatment. \n',
 'sources': 'Symptoms of cough headaches:Begin suddenly with and just after coughing or other types of straining Typically last a few seconds to a few minutes — some can last up to two hours Cause sharp, stabbing, splitting or "bursting" pain Usually affect both sides of your head and may be worse in the back of your head May be followed by a dull, aching pain for hours; At first, the flu may seem like a cold with a runny nose, sneezing and sore throat. Colds usually start slowly. But the flu tends to come on quickly. And while a cold can be miserable, you usually feel much worse with the flu.Headache. Dry, persistent cough. Shortness of breath. Tiredness and weakness. Runny or stuffy nose. Sore throat. Eye pain.'}

In [51]:
scored_docs[0]

(Document(page_content='Symptoms of cough headaches:Begin suddenly with and just after coughing or other types of straining Typically last a few seconds to a few minutes — some can last up to two hours Cause sharp, stabbing, splitting or "bursting" pain Usually affect both sides of your head and may be worse in the back of your head May be followed by a dull, aching pain for hours', metadata={'title': 'Cough headaches', 'id': 382, 'source': 'Symptoms of cough headaches:Begin suddenly with and just after coughing or other types of straining Typically last a few seconds to a few minutes — some can last up to two hours Cause sharp, stabbing, splitting or "bursting" pain Usually affect both sides of your head and may be worse in the back of your head May be followed by a dull, aching pain for hours'}),
 0.45038376911259537)